In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder. \
appName("3 Spark Read Modes"). \
getOrCreate()

In [3]:
schema_text = "order_id long, order_date date, customer_id long, order_status string"


### 1. **PERMISSIVE**
- Default mode in Spark.
- Spark attempts to parse all rows.
- Malformed rows are replaced with `null` values in the corresponding columns.
- For formats like JSON/CSV, a special `_corrupt_record` column may be added to capture the malformed data.

In [4]:
df_permissive = spark.read.csv("./dataset/orders_null.csv", header=True, schema=schema_text, mode="permissive")
df_permissive.show()

+--------+----------+-----------+---------------+
|order_id|order_date|customer_id|   order_status|
+--------+----------+-----------+---------------+
|       1|2013-07-25|      11599|         CLOSED|
|       2|2013-07-25|        256|PENDING_PAYMENT|
|       3|2013-07-25|      12111|       COMPLETE|
|       4|2013-07-25|       8827|         CLOSED|
|       5|2013-07-25|      11318|       COMPLETE|
|       6|2013-07-25|       7130|       COMPLETE|
|       7|2013-07-25|       NULL|       COMPLETE|
|       8|2013-07-25|       2911|     PROCESSING|
|       9|2013-07-25|       NULL|PENDING_PAYMENT|
|      10|2013-07-25|       5648|PENDING_PAYMENT|
+--------+----------+-----------+---------------+



### 2. **DROPMALFORMED**
- Drops all rows that are not well-formed.
- Use this mode to ignore bad or corrupted data completely.
- No additional columns are added to indicate dropped records.

In [5]:
df_dropmalformed = spark.read.csv("./dataset/orders_null.csv", header=True, schema=schema_text, mode="dropmalformed")
df_dropmalformed.show()

+--------+----------+-----------+---------------+
|order_id|order_date|customer_id|   order_status|
+--------+----------+-----------+---------------+
|       1|2013-07-25|      11599|         CLOSED|
|       2|2013-07-25|        256|PENDING_PAYMENT|
|       3|2013-07-25|      12111|       COMPLETE|
|       4|2013-07-25|       8827|         CLOSED|
|       5|2013-07-25|      11318|       COMPLETE|
|       6|2013-07-25|       7130|       COMPLETE|
|       8|2013-07-25|       2911|     PROCESSING|
|      10|2013-07-25|       5648|PENDING_PAYMENT|
+--------+----------+-----------+---------------+



### 3. **FAILFAST Mode**
- Stops processing as soon as a malformed record is encountered.
- Throws an exception if any row is not well-formed.
- Use this mode for strict validation of data quality.

In [6]:
df_failfast = spark.read.csv("./dataset/orders_null.csv", header=True, schema=schema_text, mode="failfast")
df_failfast.show()

Py4JJavaError: An error occurred while calling o60.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (DESKTOP-SGT20NN executor driver): org.apache.spark.SparkException: [MALFORMED_RECORD_IN_PARSING.WITHOUT_SUGGESTION] Malformed records are detected in record parsing: [7,15911,null,COMPLETE].
Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'. 
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInRecordParsingError(QueryExecutionErrors.scala:1611)
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:79)
	at org.apache.spark.sql.catalyst.csv.UnivocityParser$.$anonfun$parseIterator$2(UnivocityParser.scala:457)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)